In [1]:
import requests
import os
import base64
import csv
import json
import re
import pandas as pd
from datetime import datetime
from spotify_client import *
from credentials.config import *

In [3]:
def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': f'{refresh_token}'    
    }
    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)
    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token

In [4]:
playlist_token = refresh_accesss_token()

###
# imported into tkinter as a default entry in for the scrape button
# left here in case using the scrape without the gui
# replace all cases of week_num with this funciton variable: get_week_num()
###

In [5]:
def get_track_features(track_uris):
    playlist_token = refresh_accesss_token()
    track_ids = [track[14:] for track in track_uris]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
#     url = f'https://api.spotify.com/v1/audio-features?ids='
#     =7ouMYWpwJ422jRcDASZB7P%2C4VqPOruhp5EdPBeR92t6lQ%2C2takcwOaAZWiXQijPHIx7B'
    for batch in batched_tracks:
        request_data =  "%2C".join(batch)
        
        url = f'https://api.spotify.com/v1/audio-features?ids={request_data}'
        response = requests.get(
            url,
            headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
        # print(response.url)
        b = response.json()
        return print(b)

In [6]:
def get_album_tracks(album_ids):
    track_uris = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
        # print(response_json)
        tracks = response_json['items']
        for track in tracks:
            track_uris.append(track['uri'])
    get_track_features(track_uris)
    return track_uris

In [15]:
# def search_for_albums(csv_path):
artists=[]
albums=[]

albums_not_found = {'artist': [], 'album': []}
album_ids = set()


meta_16_df = pd.read_csv('albums_2016.csv')
no_nan_df = meta_16_df.dropna()
#   initialize spotify client
spotify = SpotifyAPI(client_id, client_secret)

for row in no_nan_df['album']:
    al=row
#       trim album names of suffixes (e.g. .vol 1, [ep] etc.)
    al_trim = re.split(',|\[', al)

#       search for album ids 
    temp = spotify.search({"album":al_trim[0]}, search_type="album")
    try:
        parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
    except:
#         albums_not_found['artist'].append(ar)
    albums_not_found['album'].append(al)
    album_ids.add(parse_album_ids)
# return(get_album_tracks(album_ids))
print(len(album_ids))

489


In [22]:
for albums in albums_not_found['album']:
    print(albums)

blackSUMMERS'night (2016)
Don't Let the Kids Win
"Awaken, My Love!"
One Day All of This Won't Matter Any More
We're All Gonna Die


In [16]:
no_nan_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      498 non-null    object
 1   album       498 non-null    object
 2   date        498 non-null    object
 3   week_num    498 non-null    int64 
 4   meta_score  498 non-null    int64 
 5   user_score  498 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 27.2+ KB


In [ ]:
def add_tracks_to_playlist(week_num, playlist_id):
    track_uris = [track for track in search_for_albums(week_num, csv_path)]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_uris[i * track_limit:(i + 1) * track_limit] for i in range((len(track_uris) + track_limit - 1) // track_limit)]  
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    for batch in batched_tracks:

        request_data = json.dumps(batch)

        requests.post(
            url,
            data=request_data,
            headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
    return

In [ ]:
def create_playlist(week_num):
    print(week_num)
    # week_num = get_week_num()
    request_body = json.dumps({
        'name': f'2021-week {week_num} scrape',
        'description': f'metacritic rated albums for the {week_num}th of the year',
        'public': True
    })
    url = f'https://api.spotify.com/v1/users/{spotify_user_id}/playlists'


    response = requests.post(
        url,
        data = request_body,
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'

    })
    response_json = response.json()
#     print(response_json)
    playlist_id = (response_json['id'])

    return add_tracks_to_playlist(week_num, playlist_id)